# MGST - Mikunn Galactic Search Tool
## Elite Dangerous Galaxy Search on Google Colab

This notebook allows you to run MGST searches using a galaxy database hosted on Google Drive.

**Repository:** https://github.com/kaw97/MGST

**Database:** Download from https://drive.google.com/drive/folders/1m4Fl14xxaEm5rY9uoo9w9BKsAj7gHq_a

## 1. Setup: Install MGST and Dependencies

In [ ]:
# Install MGST from GitHub
# Using --no-cache-dir to ensure fresh install
!pip install --no-cache-dir git+https://github.com/kaw97/MGST.git

# Verify installation
import sys
try:
    import mgst
    print(f"\n✓ MGST installed successfully")
    print(f"  Version: {mgst.__version__ if hasattr(mgst, '__version__') else 'dev'}")
    print(f"  Location: {mgst.__file__}")
    
    # Verify submodules
    import mgst.cli
    import mgst.core
    import mgst.data
    print(f"  ✓ All submodules loaded")
    
    # Test the CLI
    print(f"\nCLI Help:")
    !mgst --help
except ImportError as e:
    print(f"\n✗ Installation failed: {e}")
    print(f"\nTroubleshooting:")
    print(f"  1. Try restarting the runtime (Runtime > Restart runtime)")
    print(f"  2. Then run this cell again")
    sys.exit(1)

## 2. Mount Google Drive

Mount your Google Drive to access the galaxy database files.

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Set the path to your database directory
# MODIFY THIS PATH to point to your database folder in Google Drive
DATABASE_PATH = '/content/drive/MyDrive/galaxy_sectors_compressed'

# Verify database exists
if os.path.exists(DATABASE_PATH):
    print(f"✓ Database found at: {DATABASE_PATH}")
    # List a few sector files
    sectors = [f for f in os.listdir(DATABASE_PATH) if f.endswith('.jsonl.gz')][:5]
    print(f"\nSample sectors found: {len(sectors)}")
    for sector in sectors:
        print(f"  - {sector}")
else:
    print(f"⚠ Database not found at: {DATABASE_PATH}")
    print("\nPlease update DATABASE_PATH to point to your galaxy database folder.")
    print("\nExample paths:")
    print("  /content/drive/MyDrive/galaxy_sectors_compressed")
    print("  /content/drive/MyDrive/MGST/galaxy_sectors")

## 3. Create Output Directory

In [ ]:
# Create output directory
OUTPUT_DIR = '/content/mgst_results'
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Output directory: {OUTPUT_DIR}")

## 4. Create Search Patterns

Define your search patterns as JSON files.

In [ ]:
import json

# Create patterns directory
PATTERNS_DIR = '/content/patterns'
os.makedirs(PATTERNS_DIR, exist_ok=True)

# Example 1: Supply Hub Pattern (ELW/Water World with rocky moon)
supply_hub_pattern = {
    "description": "Supply Hub Candidates - ELW or Water World with rocky moon",
    "name": "*",
    "bodies": [
        {
            "comment": "Match ELW or Water World (the parent planet)",
            "subType": ["Earth-like world", "Water world"]
        },
        {
            "comment": "Match rocky moon orbiting the ELW/Water World",
            "subType": "Rocky body",
            "parents": [{"Planet": "*"}]
        }
    ]
}

# Save supply hub pattern
with open(f'{PATTERNS_DIR}/supply_hub.json', 'w') as f:
    json.dump(supply_hub_pattern, f, indent=2)

# Example 2: All Systems Pattern (for testing)
all_systems_pattern = {
    "description": "Match all systems",
    "name": "*"
}

with open(f'{PATTERNS_DIR}/all_systems.json', 'w') as f:
    json.dump(all_systems_pattern, f, indent=2)

# Example 3: Interesting Systems (high-value exploration)
interesting_pattern = {
    "description": "High-value exploration targets",
    "name": "*",
    "bodies": [
        {
            "comment": "Earth-like worlds",
            "subType": "Earth-like world"
        }
    ]
}

with open(f'{PATTERNS_DIR}/interesting_systems.json', 'w') as f:
    json.dump(interesting_pattern, f, indent=2)

print("✓ Pattern files created:")
for pattern in os.listdir(PATTERNS_DIR):
    print(f"  - {pattern}")

## 5. Run Searches

Now you can run different types of searches. Choose the search mode that fits your needs.

### 5.1 Corridor Search

Search for systems along a corridor between two coordinates.

In [ ]:
# Corridor Search Example
# Modify these parameters for your search

START_COORDS = "0,0,0"  # Starting coordinates (X,Y,Z)
END_COORDS = "1000,0,0"  # Ending coordinates (X,Y,Z)
RADIUS = 500  # Corridor radius in light years
PATTERN_FILE = f"{PATTERNS_DIR}/supply_hub.json"
OUTPUT_FILE = f"{OUTPUT_DIR}/corridor_search.jsonl"
WORKERS = 4  # Number of parallel workers (adjust based on your needs)

!mgst filter \
    --mode corridor \
    --start "{START_COORDS}" \
    --end "{END_COORDS}" \
    --radius {RADIUS} \
    --pattern-file "{PATTERN_FILE}" \
    --database "{DATABASE_PATH}" \
    --output "{OUTPUT_FILE}" \
    --workers {WORKERS}

### 5.2 Sector Search

Search for systems within specific sectors.

In [ ]:
# Sector Search Example
# Modify these parameters for your search

SECTORS = "Lagoon_Sector,Trifid_Sector"  # Comma-separated sector names
PATTERN_FILE = f"{PATTERNS_DIR}/interesting_systems.json"
OUTPUT_FILE = f"{OUTPUT_DIR}/sector_search.tsv"
WORKERS = 4

!mgst filter \
    --mode sectors \
    --sectors "{SECTORS}" \
    --pattern-file "{PATTERN_FILE}" \
    --database "{DATABASE_PATH}" \
    --output "{OUTPUT_FILE}" \
    --workers {WORKERS} \
    --format tsv

### 5.3 Galaxy-Wide Search

Search the entire galaxy (WARNING: This can take a very long time!)

In [ ]:
# Galaxy Search Example (use with caution - very resource intensive)
# Uncomment to run

# PATTERN_FILE = f"{PATTERNS_DIR}/supply_hub.json"
# OUTPUT_FILE = f"{OUTPUT_DIR}/galaxy_search.jsonl"
# WORKERS = 4

# !mgst filter \
#     --mode galaxy \
#     --pattern-file "{PATTERN_FILE}" \
#     --database "{DATABASE_PATH}" \
#     --output "{OUTPUT_FILE}" \
#     --workers {WORKERS}

### 5.4 Dry Run (Test Before Executing)

See what would be searched without actually running the search.

In [ ]:
# Dry run to test your search parameters
!mgst filter \
    --mode corridor \
    --start "0,0,0" \
    --end "1000,0,0" \
    --radius 100 \
    --pattern-file "{PATTERNS_DIR}/all_systems.json" \
    --database "{DATABASE_PATH}" \
    --dry-run

## 6. View Results

Load and display search results.

In [ ]:
import pandas as pd
import json

# Function to load JSONL results
def load_jsonl_results(filepath):
    """Load results from JSONL file"""
    if not os.path.exists(filepath):
        print(f"File not found: {filepath}")
        return None
    
    results = []
    with open(filepath, 'r') as f:
        for line in f:
            results.append(json.loads(line))
    
    return pd.DataFrame(results)

# Function to load TSV results
def load_tsv_results(filepath):
    """Load results from TSV file"""
    if not os.path.exists(filepath):
        print(f"File not found: {filepath}")
        return None
    
    return pd.read_csv(filepath, sep='\t')

# Example: Load and display results
# Modify the filename to match your output
results_file = f"{OUTPUT_DIR}/corridor_search.jsonl"

if os.path.exists(results_file):
    if results_file.endswith('.jsonl'):
        df = load_jsonl_results(results_file)
    else:
        df = load_tsv_results(results_file)
    
    if df is not None:
        print(f"Found {len(df)} matching systems\n")
        print("First 5 results:")
        display(df.head())
else:
    print(f"No results file found. Run a search first!")

## 7. Download Results

Download results to your local machine or save to Google Drive.

In [ ]:
from google.colab import files
import shutil

# Option 1: Download to local machine
def download_results(filepath):
    """Download a file to local machine"""
    if os.path.exists(filepath):
        files.download(filepath)
        print(f"Downloaded: {filepath}")
    else:
        print(f"File not found: {filepath}")

# Download the results file
# download_results(f"{OUTPUT_DIR}/corridor_search.jsonl")

# Option 2: Copy results to Google Drive for persistence
def save_to_drive(source_dir, drive_destination):
    """Copy results directory to Google Drive"""
    if os.path.exists(source_dir):
        os.makedirs(os.path.dirname(drive_destination), exist_ok=True)
        shutil.copytree(source_dir, drive_destination, dirs_exist_ok=True)
        print(f"Saved results to: {drive_destination}")
    else:
        print(f"Source directory not found: {source_dir}")

# Save all results to Google Drive
# save_to_drive(OUTPUT_DIR, '/content/drive/MyDrive/MGST_Results')

print("Use the functions above to download or save your results.")

## 8. Custom Pattern Creator

Create your own custom search patterns interactively.

In [ ]:
# Create a custom pattern
def create_custom_pattern(description, name="*", bodies=None):
    """Helper function to create custom search patterns"""
    pattern = {
        "description": description,
        "name": name
    }
    
    if bodies:
        pattern["bodies"] = bodies
    
    return pattern

# Example: Create a pattern for ammonia worlds with rings
ammonia_rings_pattern = create_custom_pattern(
    description="Ammonia worlds with ring systems",
    bodies=[
        {
            "comment": "Ammonia world",
            "subType": "Ammonia world"
        },
        {
            "comment": "Has rings",
            "type": "Ring"
        }
    ]
)

# Save custom pattern
pattern_name = "ammonia_rings"
with open(f'{PATTERNS_DIR}/{pattern_name}.json', 'w') as f:
    json.dump(ammonia_rings_pattern, f, indent=2)

print(f"Created custom pattern: {pattern_name}.json")
print(json.dumps(ammonia_rings_pattern, indent=2))

## 9. Common Search Recipes

Pre-configured searches for common use cases.

In [ ]:
# Recipe 1: Find colonization candidates near Sol
print("Recipe 1: Colonization candidates near Sol")
print("Searching for Earth-like worlds within 500 LY of Sol...\n")

# !mgst filter \
#     --mode corridor \
#     --start "0,0,0" \
#     --end "0,0,0" \
#     --radius 500 \
#     --pattern-file "{PATTERNS_DIR}/interesting_systems.json" \
#     --database "{DATABASE_PATH}" \
#     --output "{OUTPUT_DIR}/colonization_candidates.jsonl" \
#     --workers 4

# Recipe 2: Route planning between two stations
print("\nRecipe 2: Route planning (example coordinates)")
print("Finding supply hubs along travel route...\n")

# !mgst filter \
#     --mode corridor \
#     --start "-468,-92,4474" \
#     --end "-575,-37,5142" \
#     --radius 500 \
#     --pattern-file "{PATTERNS_DIR}/supply_hub.json" \
#     --database "{DATABASE_PATH}" \
#     --output "{OUTPUT_DIR}/route_supply_hubs.jsonl" \
#     --workers 4

print("Uncomment the commands above to run these recipes.")

## Tips and Notes

1. **Database Location**: Make sure your database path points to the correct folder containing `.jsonl.gz` files
2. **Performance**: Adjust `--workers` based on available resources (Google Colab has limitations)
3. **Output Formats**: Use `--format tsv` for spreadsheet-compatible output, or `jsonl` for structured data
4. **Logs**: Each search automatically creates comprehensive logs in subdirectories
5. **Memory**: Galaxy-wide searches are very memory intensive - use corridor or sector searches when possible
6. **Pattern Files**: See the repository's `patterns/` directory for more examples
7. **Coordinates**: Use EDSM or Spansh to get system coordinates for corridor searches

## Resources

- **GitHub Repository**: https://github.com/kaw97/MGST
- **Database Download**: https://drive.google.com/drive/folders/1m4Fl14xxaEm5rY9uoo9w9BKsAj7gHq_a
- **EDSM**: https://www.edsm.net/ (for system coordinates)
- **Spansh**: https://spansh.co.uk/ (for galaxy data and route planning)